### Mencoba membuat RAG lokal dengan llama2 70B (CPU core I5).

Kita buat file .env untuk simpan credential API dll selanjutnya declare model, tak hanya untuk llama2 tapi juga model OpenAI (melalui API) sebagai pembanding output <br>

In [9]:
import os
from dotenv import load_dotenv

MODEL = "llama2"

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_KEY")
# MODEL = "gpt-3.5-turbo"

In [17]:
#from langchain_openai.chat_models import ChatOpenAI
# from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

# model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Give me a motivating quote")

'\n"Believe you can and you\'re halfway there." - Theodore Roosevelt\n\nThis quote emphasizes the importance of self-belief and confidence in achieving our goals. It suggests that by having faith in ourselves and our abilities, we can overcome obstacles and reach our full potential. Remember, belief is the foundation of success, so have faith in yourself and your dreams!'

Parsing respon dari model (untuk respon dari model GPT OPENAI)

In [16]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Give me a motivating quote")

'\n"Believe you can and you\'re halfway there." - Theodore Roosevelt\n\nThis quote emphasizes the importance of belief in achieving success. It suggests that having faith in yourself and your abilities is crucial to making progress towards your goals. By believing in yourself, you are more likely to take action and work towards your objectives, which can ultimately lead to success. So, no matter what challenges you may be facing, remember this quote and have faith in yourself!'

Membuat template percakapan dengan menentukan context dan pertanyaan

In [12]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "Maaf, terkait ini Saya belum dapat memberikan Anda jawaban".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "Maaf, terkait ini Saya belum dapat memberikan Anda jawaban".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [22]:
chain = prompt | model | parser

chain.invoke({"context": "Semua orang menyapa saya dengan panggilan Jhony Iskandar", 
"question": "Siapa namamu untuk saya sapa jika nanti kita bertemu'?"})

"Maaf, terkait ini Saya belum dapat memberikan Anda jawaban. As a virtual assistant, I don't have access to personal information or the ability to make assumptions about individuals based on their names or any other personal details. It is important to respect people's privacy and only share information that is publicly available or has been explicitly shared by the individual themselves. Is there anything else I can help you with?"

Load PDF dan split tiap lembar

In [19]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./letterC.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': './letterC.pdf', 'page': 0}, page_content='Jurnal Ilm iah FENOMENA, Volume X V, Nomor 2, November 2017: 1665 -1673  \n \n \n  \n 1665  \n FUNGSI BUKU LETTER C DESA  \nDALAM KAITANNYA DENGAN BUKU KERAWANGAN DESA  \nSEBAGAI LANDASAN YURIDIS DAN ALAT BUKTI AWAL  \nATAS PENGAKUAN KEPEMILIKAN HAK ATAS TANAH  \n \nOleh :  Tedjo Asmo Sugeng, S.H., M.H.*   1 \n \nABSTRAK  \nPengakuan Kepemilikan Hak Atas Tanah, dimana pun tanah itu \nberada dan siapapun pemiliknya maka akan selalu ada dan \ntercatat dalam riwayat tanah pada catatan  Buku Tanah Desa , yang \nlebih dikenal ditiap -tiap Kantor Desa/Kantor Kelurahan  yaitu \ndengan sebutan buku Letter C Desa dan Buku Kerawangan Desa.  \nKedua Buku Tanah Desa tersebut sudah ada pada Zaman India – \nBelanda dan sampai sekarang tetap dipakai sebagai Dokumen \nNegara , yang disimpan di tiap -tiap Kantor Desa/Kantor \nKelurahan . Kedua Buku Tanah Desa tersebut selalu berdampingan \nkarena saling melengkap i.  \nSebelum ber

In [20]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [21]:
retriever = vectorstore.as_retriever()
retriever.invoke("letterC")

[Document(metadata={'source': './letterC.pdf', 'page': 8}, page_content='Jurnal Ilm iah FENOMENA, Volume X V, Nomor 2, November 2017: 1665 -1673  \n \n \n  \n 1673  \n 5. PENUTUP  \nFungsi Buku Letter C Desa dalam kaitannya dengan Buku Kerawangan \nDesa sebagai Dokumen Negara yan g dapat berfungsi sebagai landasan yuridis \ndan acuan dalam setiap perubahan pada peralihan hak milik atas tanah \nkepada pihak lainnya . Pemegang hak milik atas tanah yang berupa Surat \nKutipan Letter C Desa atau Surat Koher Petok D adalah dapat dijadikan dasa r \nbukti awal dalam pengajuan permohonan Sertifikat Tanah pada Kantor \nBadan Pertanahan  \n   \n6. DAFTAR PUSTAKA  \n \nAndi Hamzah 1986, Kamus Hukum. Ghalia Indonesia. Jakarta  \n \nAndreas Lumme. 2001, Majalah Hukum Pro Justitia, Edisi Kedua Tahun XIX, \nFakultas hukum. un iversitas Parahyangan. Bandung  \n \nAP. Perlindungan II 2001, Pendaftaran Tanah di Indonesia, Cet Pertama \nMandar Maju. Bandung  \n \nBoedi Harsono. 1999, Hukum Agraria Nasion